In [17]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env.local')
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
import os
import json


In [18]:

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Create a Pinecone index
pc.create_index(
    name="rag",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

In [ ]:
# Load the review data
data = json.load(open("reviews.json"))
data['reviews']

In [ ]:
processed_data = []
client = OpenAI()

